In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

In [3]:
conf = pyspark.SparkConf().setAppName("SparkMaster").setMaster("local")
sc = SparkSession.builder.config(conf=conf).config("spark.sql.legacy.timeParserPolicy", "LEGACY").getOrCreate()

25/02/18 16:31:52 WARN Utils: Your hostname, shasankperiwal resolves to a loopback address: 127.0.1.1; using 192.168.20.45 instead (on interface wlp0s20f3)
25/02/18 16:31:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/18 16:31:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 60170)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/opt/spark/spark-3.4.4-bin-hadoop3/python/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/opt/spark/spark-3.4.4-bin-hadoop3/python/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/opt/spark/spark-3.4.4-bin-hadoop3/python/pyspark/accumulators.py", line 257, in accum_updates
    num_updates = read_int(self.rfile)
  

In [4]:
column = ["category", "brand_name"]
data = [
    ("chocolates", "5-star"),
	("", "dairy milk"),
	("", "perk"),
	("", "eclair"),
    ("Biscuits",	"britannia"),
	("", "good day"),
	("", "boost")
]
df = sc.createDataFrame(data=data, schema=column)

# OUTPUT:
# +----------+----------+---+----+
# |  category|brand_name| rn|  ld|
# +----------+----------+---+----+
# |chocolates|    5-star|  0|   4|
# |  Biscuits| britannia|  4|9999|
# +----------+----------+---+----+

In [26]:
df = df.withColumn("rn", monotonically_increasing_id())
df2= df.filter("len(CATEGORY) > 0").withColumn("ld", lead("rn", 1, 9999).over(Window.orderBy("rn")))
df.show()
df2.show()

# df.show()
# df = df.withColumn("id", monotonically_increasing_id())
# df = df.withColumn("rn", sum(when(col("category") != "", 1)).over(Window.orderBy("id")))
# cdf = df.filter(col("category") != "")
# df.alias("a").join(cdf.alias("b"), on="rn", how="left").select(col("b.category"), col("a.brand_name")).show()

+----------+----------+---+---+
|  category|brand_name| id| rn|
+----------+----------+---+---+
|chocolates|    5-star|  0|  0|
|          |dairy milk|  1|  1|
|          |      perk|  2|  2|
|          |    eclair|  3|  3|
|  Biscuits| britannia|  4|  4|
|          |  good day|  5|  5|
|          |     boost|  6|  6|
+----------+----------+---+---+



25/02/18 17:17:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 17:17:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 17:17:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 17:17:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 17:17:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+----------+---+---+----+
|  category|brand_name| id| rn|  ld|
+----------+----------+---+---+----+
|chocolates|    5-star|  0|  0|   4|
|  Biscuits| britannia|  4|  4|9999|
+----------+----------+---+---+----+



In [ ]:
# df2_renamed = df2.withColumnRenamed("rn", "df2rn")
# Please alias the Datasets with different names via `Dataset.as` before joining them, and specify the column using qualified name, 
# e.g. `df.as("a").join(df.as("b"), $"a.id" > $"b.id")`.
# df.join(df2_renamed, (col("rn").between(col("df2rn"), col("ld"))), how="inner").select(df2_renamed["category"], df["brand_name"]).show()
# OUTPUT
# +----------+----------+
# |  category|brand_name|
# +----------+----------+
# |chocolates|    5-star|
# |chocolates|dairy milk|
# |chocolates|      perk|
# |chocolates|    eclair|
# |chocolates| britannia|
# |  Biscuits| britannia|
# |  Biscuits|  good day|
# |  Biscuits|     boost|
# +----------+----------+
df.alias("a").join(df2.alias("b"), (col("a.rn").between(col("b.rn"), col("ld"))), how="inner").select(col("b.category"), col("a.brand_name")).show()

19_Rolling Balances

In [27]:
schema = StructType([
    StructField("user_id", IntegerType()),
    StructField("event_name", StringType()),
    StructField("event_date", DateType()),
    StructField("country", StringType()),
])

data = [
    (1, 'app-installed', datetime.strptime('2022-01-01', '%Y-%m-%d').date(), 'India'),
    (1, 'app-purchase', datetime.strptime('2022-01-02', '%Y-%m-%d').date(), 'India'),
    (2, 'app-installed', datetime.strptime('2022-01-01', '%Y-%m-%d').date(), 'USA'),
    (3, 'app-installed', datetime.strptime('2022-01-01', '%Y-%m-%d').date(), 'USA'),
    (3, 'app-purchase', datetime.strptime('2022-01-03', '%Y-%m-%d').date(), 'USA'),
    (4, 'app-installed', datetime.strptime('2022-01-03', '%Y-%m-%d').date(), 'India'),
    (4, 'app-purchase', datetime.strptime('2022-01-03', '%Y-%m-%d').date(), 'India'),
    (5, 'app-installed', datetime.strptime('2022-01-03', '%Y-%m-%d').date(), 'SL'),
    (5, 'app-purchase', datetime.strptime('2022-01-03', '%Y-%m-%d').date(), 'SL'),
    (6, 'app-installed', datetime.strptime('2022-01-04', '%Y-%m-%d').date(), 'Pakistan'),
    (6, 'app-purchase', datetime.strptime('2022-01-04', '%Y-%m-%d').date(), 'Pakistan'),
]

In [28]:
rb = sc.createDataFrame(data=data, schema=schema)
rb.show()

+-------+-------------+----------+--------+
|user_id|   event_name|event_date| country|
+-------+-------------+----------+--------+
|      1|app-installed|2022-01-01|   India|
|      1| app-purchase|2022-01-02|   India|
|      2|app-installed|2022-01-01|     USA|
|      3|app-installed|2022-01-01|     USA|
|      3| app-purchase|2022-01-03|     USA|
|      4|app-installed|2022-01-03|   India|
|      4| app-purchase|2022-01-03|   India|
|      5|app-installed|2022-01-03|      SL|
|      5| app-purchase|2022-01-03|      SL|
|      6|app-installed|2022-01-04|Pakistan|
|      6| app-purchase|2022-01-04|Pakistan|
+-------+-------------+----------+--------+



In [29]:
superstoreschema = StructType([
    StructField("Row ID",   IntegerType()),
    StructField("Order ID", StringType()),
    StructField("Order Date",   DateType()),
    StructField("Ship Date",    StringType()),
    StructField("Ship Mode",    StringType()),
    StructField("Customer ID",  StringType()),
    StructField("Customer Name",    StringType()),
    StructField("Segment",  StringType()),
    StructField("Country",  StringType()),
    StructField("City", StringType()),
    StructField("State",    StringType()),
    StructField("Postal Code",  StringType()),
    StructField("Region",   StringType()),
    StructField("Product ID",   StringType()),
    StructField("Category", StringType()),
    StructField("Sub-Category", StringType()),
    StructField("Product Name", StringType()),
    StructField("Sales",    DoubleType()),
    StructField("Quantity", IntegerType()),
    StructField("Discount", DoubleType()),
    StructField("Profit",   DoubleType())
])

In [30]:
ss = sc.read.csv("assets/Superstore.csv", schema=superstoreschema, header=True, dateFormat="MM/dd/yyyy")

In [31]:
ss.show(5)

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156|2016-11-08|11/11/2016|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset 

In [41]:
ss.select("Order ID", "Order Date", coalesce("sales", lit(0)).alias("sales")).withColumn("rolling_blncs", sum("sales").over(Window.orderBy(col("Order Date")).rowsBetween(Window.unboundedPreceding, 0))).orderBy("Order Date").show()

25/02/18 17:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 17:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 17:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 17:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 17:27:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------+----------+-------+------------------+
|      Order ID|Order Date|  sales|     rolling_blncs|
+--------------+----------+-------+------------------+
|CA-2014-103800|2014-01-03|    0.0|               0.0|
|CA-2014-112326|2014-01-04| 11.784|            11.784|
|CA-2014-112326|2014-01-04|272.736|            284.52|
|CA-2014-112326|2014-01-04|   3.54|            288.06|
|CA-2014-141817|2014-01-05| 19.536|           307.596|
|CA-2014-130813|2014-01-06|  19.44|           327.036|
|CA-2014-106054|2014-01-06|  12.78|           339.816|
|CA-2014-167199|2014-01-06|2573.82|          2913.636|
|CA-2014-167199|2014-01-06| 609.98|          3523.616|
|CA-2014-167199|2014-01-06|   5.48|          3529.096|
|CA-2014-167199|2014-01-06| 391.98|          3921.076|
|CA-2014-167199|2014-01-06| 755.96|          4677.036|
|CA-2014-167199|2014-01-06|  31.12|          4708.156|
|CA-2014-167199|2014-01-06|   6.54|          4714.696|
|CA-2014-105417|2014-01-07| 76.728|          4791.424|
|CA-2014-1

In [45]:
#  CREATE TABLE event_status(event_time varchar(10),status varchar(10));
timeschema = StructType([
    StructField("event_time", StringType()),
    StructField("status", StringType())
])

data = [
    ('10:01','on'),
    ('10:02','on'),
    ('10:03','on'),
    ('10:04','off'),
    ('10:07','on'),
    ('10:08','on'),
    ('10:09','off'),
    ('10:11','on'),
    ('10:12','off')
]


In [46]:
ts = sc.createDataFrame(data=data, schema=timeschema)
ts.show()
# OUTPUT
# +---+----------+--------+------+
# |rnk|start_time|end_time|status|
# +---+----------+--------+------+
# |  0|     10:01|   10:04|    ON|
# |  1|     10:07|   10:09|    ON|
# |  2|     10:11|   10:12|    ON|
# +---+----------+--------+------+

+----------+------+
|event_time|status|
+----------+------+
|     10:01|    on|
|     10:02|    on|
|     10:03|    on|
|     10:04|   off|
|     10:07|    on|
|     10:08|    on|
|     10:09|   off|
|     10:11|    on|
|     10:12|   off|
+----------+------+



In [61]:
# ts = ts.withColumn("nxt", lag("status", 1).over(Window.orderBy(col("event_time")))).\
# withColumn("rn", sum(when((col("status") == "on") & (col("nxt") == "off"), 1).otherwise(0)).over(Window.orderBy(col("event_time"))))
# ts.groupBy("rn").agg(min("event_time").alias("start_time"), max("event_time").alias("end_time")).select("start_time", "end_time", lit("ON")).show()

+----------+--------+---+
|start_time|end_time| ON|
+----------+--------+---+
|     10:01|   10:04| ON|
|     10:07|   10:09| ON|
|     10:11|   10:12| ON|
+----------+--------+---+



25/02/18 18:01:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 18:01:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 18:01:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 18:01:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 18:01:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 18:01:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/18 1

25/02/18 18:47:16 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2572543 ms exceeds timeout 120000 ms
25/02/18 18:47:16 WARN SparkContext: Killing executors is not supported by current scheduler.
25/02/18 18:47:20 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.

In [ ]:
ts = ts.withColumn("lgstts", coalesce(lag("status", 1).over(Window.orderBy(col("event_time"))), col("status"))) \
    .withColumn("rnk", sum(when((col("status")=="on") & (col("lgstts") == "off"), 1).otherwise(0)).over(Window.orderBy(col("event_time"))))

In [ ]:
ts.groupBy("rnk").agg(min("event_time").alias("start_time"), max("event_time").alias("end_time")).withColumn("status", lit("ON")).show()

In [ ]:
tblschema = StructType([
    StructField("trade_id", StringType()),
    StructField("trade_timestamp", StringType()),
    StructField("trade_stock", StringType()),
    StructField("quantity", IntegerType()),
    StructField("price", IntegerType())
])

data = [('TRADE1','10:01:05','ITJunction4All',100,20),('TRADE2','10:01:06','ITJunction4All',20,15),('TRADE3','10:01:08','ITJunction4All',150,30),('TRADE4','10:01:09','ITJunction4All',300,32),('TRADE5','10:10:00','ITJunction4All',-100,19),('TRADE6','10:10:01','ITJunction4All',-300,19)]
tradedf = sc.createDataFrame(data=data, schema=tblschema)


In [ ]:
tradedf = tradedf.withColumn("trade_timestamp", to_timestamp(tradedf["trade_timestamp"], 'HH:mm:ss'))

In [ ]:
result = tradedf.alias("a").join(
    tradedf.alias("b"),
    on=(
        col("a.trade_timestamp").between(
            col("b.trade_timestamp") - expr("INTERVAL 10 SECONDS"),
            col("b.trade_timestamp") + expr("INTERVAL 10 SECONDS")
        ) & (col("a.trade_id") != col("b.trade_id"))
    ),
    how="inner"
).select(
    when(col("a.trade_id") > col("b.trade_id"), col("a.trade_id")).otherwise(col("b.trade_id")).alias("t1_trade_id"), 
    when(col("a.trade_id") <= col("b.trade_id"), col("a.trade_id")).otherwise(col("b.trade_id")).alias("t2_trade_id"), 
    when(col("a.trade_id") > col("b.trade_id"), col("a.trade_timestamp")).otherwise(col("b.trade_timestamp")).alias("t1_trade_timestamp"), 
    when(col("a.trade_id") <= col("b.trade_id"), col("a.trade_timestamp")).otherwise(col("b.trade_timestamp")).alias("t2_trade_timestamp"), 
    when(col("a.trade_id") > col("b.trade_id"), col("a.price")).otherwise(col("b.price")).alias("t1_price"), 
    when(col("a.trade_id") <= col("b.trade_id"), col("a.price")).otherwise(col("b.price")).alias("t2_price"), 
    
)
result.show()

In [ ]:
result.select("*").filter((abs(col("t1_price")-col("t2_price"))/(col("t1_price")+col("t2_price"))*100)>10).show()